In [19]:
import langchain
print(langchain.__version__)

0.3.23


In [20]:
from langchain.agents import initialize_agent, Tool
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

In [21]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun

agent_search_tool = Tool(
    name="Duck Duck go Search",
    func=DuckDuckGoSearchRun().run,
    description="Use this tool to search on web if answer is not present in Knowlwdge Base of LLM"
)

agent_llm = ChatGroq(
    model="llama3-70b-8192"
)

ddg_tool = [agent_search_tool]

search_agent = initialize_agent(ddg_tool, agent_llm, agent="zero-shot-react-description", verbose=True)
agent_response = search_agent.run("What are the latest movies that will be release in April 2025")

print(agent_response)



> Entering new AgentExecutor chain...
Thought: I need to find the latest information on movies releasing in April 2025. 

Action: Duck Duck go Search
Action Input: "upcoming movies april 2025"
Observation: Here is our list of every movie releasing in theaters in April 2025. We have some huge blockbusters coming to theaters next month. ... A Minecraft Movie will be released in theaters on April 4, 2025. April 2025 will bring a dose of horror with a vampire movie and a new take on Mickey Mouse, a movie based on a popular video game, and a lot more. Horror, fantasy, and a lot more. ... Every Movie Coming To Theaters In April 2025. By Adrienne Tyler. Published Mar 28, 2025. Upcoming Movies in April (2025) (Images via Amazon MGM Studios, Lionsgate Movies, 20th Century Studios, Netflix) Every movie enthusiast will appreciate the upcoming 2025 movie season because several films will appear on theater screens and streaming services. The upcoming films will feature supernatural horror and act

In [23]:
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper

wikipedia_search_tool = Tool(
    name="Wikipedia Search",
    func=WikipediaAPIWrapper().run,
    description="Use this tool to search on Wikipedia if answer is not present in Knowledge Base of LLM"
)
agent_llm = ChatGroq(
    model="llama3-70b-8192"
)
wiki_search_tool = [wikipedia_search_tool]
search_agent = initialize_agent(wiki_search_tool, agent_llm, agent="zero-shot-react-description", verbose=True)
agent_response = search_agent.run("Give me the information about tesla car")
print(agent_response)



> Entering new AgentExecutor chain...
Thought: I don't have enough information about Tesla cars in my Knowledge Base, so I'll search on Wikipedia to get more information.

Action: Wikipedia Search
Action Input: "Tesla, Inc."
Observation: Page: Tesla, Inc.
Summary: Tesla, Inc. ( TEZ-lə or   TESS-lə) is an American multinational automotive and clean energy company. Headquartered in Austin, Texas, it designs, manufactures and sells battery electric vehicles (BEVs), stationary battery energy storage devices from home to grid-scale, solar panels and solar shingles, and related products and services.
Tesla was incorporated in July 2003 by Martin Eberhard and Marc Tarpenning as Tesla Motors. Its name is a tribute to inventor and electrical engineer Nikola Tesla. In February 2004, Elon Musk led Tesla's first funding round and became the company's chairman; in 2008, he was named chief executive officer. In 2008, the company began production of its first car model, the Roadster sports car, fol